### Main notebook

In [4]:
from scraper import DataLoader

### Scrape google map's reviews into dataframe

In [3]:
# insert goole maps business url
url = 'https://maps.app.goo.gl/geQwCeu2riPkrSqt8'


In [5]:
# Loader init
loader = DataLoader(url)

In [6]:
# navigate to the reviews page
loader.navigate()

In [7]:
loader.create_reviews_df()
loader.df

the type of each review is: <class 'selenium.webdriver.remote.webelement.WebElement'>


,text_review,stars,time
0,הגענו אני ובעלי לחגוג לו יום הולדת במקום.\nצוו...,5,לפני 5 חודשים
1,הגענו אני ובעלי לחגוג לו יום הולדת במקום.\nצוו...,5,לפני 5 חודשים
2,זו לא הפעם הראשונה שאני מבקרת ברוסטיקו אך כל פ...,5,לפני שנה
3,זו לא הפעם הראשונה שאני מבקרת ברוסטיקו אך כל פ...,5,לפני שנה
4,"אוכל מאוד טעים ואווירה נהדרת..ישבנו בחוץ, ולמ...",5,לפני 8 חודשים
5,"אוכל מאוד טעים ואווירה נהדרת..ישבנו בחוץ, ולמ...",5,לפני 8 חודשים
6,האוכל היה מצוין אבל השרות איטי להכעיס. לפחות ה...,4,לפני 8 חודשים
7,האוכל היה מצוין אבל השרות איטי להכעיס. לפחות ה...,4,לפני 8 חודשים
8,"מקום חמוד, חלק מהאוכל היה מעולה וחלק טעון שיפו...",3,לפני שנה
9,"מקום חמוד, חלק מהאוכל היה מעולה וחלק טעון שיפו...",3,לפני שנה
